In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context('notebook', font_scale=1.5)

In [2]:
import warnings
warnings.simplefilter('ignore', FutureWarning)

**1**. (25 points)

In this exercise, we will practice using Pandas dataframes to explore and summarize a data set `heart`.

This data contains the survival time after receiving a heart transplant, the age of the patient and whether or not the survival time was censored

- Number of Observations - 69
- Number of Variables - 3

Variable name definitions::

- survival - Days after surgery until death
- censors - indicates if an observation is censored. 1 is uncensored
- age - age at the time of surgery

Answer the following questions (5 points each) with respect to the `heart` data set:

- How many patients were censored?
- What is the correlation coefficient between age and survival for uncensored patients? 
- What is the average age for censored and uncensored patients?
- What is the average survival time for censored and uncensored patients under the age of 45?
- What is the survival time of the youngest and oldest uncensored patient?


In [3]:
import statsmodels.api as sm
heart = sm.datasets.heart.load_pandas().data
heart.head(n=6)

,survival,censors,age
0,15.0,1.0,54.3
1,3.0,1.0,40.4
2,624.0,1.0,51.0
3,46.0,1.0,42.5
4,127.0,1.0,48.0
5,64.0,1.0,54.6


In [4]:
# How many patients were censored?

sum(heart.censors == 0)

24

In [5]:
# What is the correlation coefficient between age and survival for uncensored patients?

uncensored = heart[heart.censors == 1]
np.corrcoef(uncensored.age, uncensored.survival)[0,1]

0.003256499283211926

In [6]:
# What is the average age for censored and uncensored patients?

heart.groupby('censors')['age'].mean()

censors
0.0    41.729167
1.0    48.484444
Name: age, dtype: float64

In [7]:
# What is the average survival time for censored and uncensored patients under the age of 45?

young = heart[heart.age < 45]
young.groupby('censors')['survival'].mean()

censors
0.0    712.818182
1.0    169.909091
Name: survival, dtype: float64

In [8]:
# What is the survival time of the youngest and oldest uncensored patient?

uncensored.survival[np.argmin(uncensored.age)], uncensored.survival[np.argmax(uncensored.age)]

(228.0, 60.0)

**2**. (35 points)

- Consider a sequence of $n$ Bernoulli trials with success probabilty $p$ per trial. A string of consecutive successes is known as a success *run*. Write a function that returns the counts for runs of length $k$ for each $k$ observed in a dictionary. (10 points)

For example: if the trials were [0, 1, 0, 1, 1, 0, 0, 0, 0, 1], the function should return 
```
{1: 2, 2: 1})
```

Test that it does so.

- What is the probability of observing at least one run of length 5 or more when $n=100$ and $p=0.5$?. Estimate this from 100,000 simulated experiments. Is this more, less or equally likely than finding runs of length 7 or more when $p=0.7$? (10 points)

- There is an exact solution

$$
s_n = \sum_{i=1}^n{f_i} \\
f_n = u_n - \sum_{i=1}^{n-1} {f_i u_{n-i}} \\
u_n = p^k - \sum_{i=1}^{k-1} u_{n-i} p^i
$$

Implement the exact solution using caching to avoid re-calculations and calculate the same two probabilities found by simulation. (15 points)

**Part 1**

In [ ]:
from collections import Counter

def count_runs(xs):
    """Count number of success runs of length k."""
    ys = []
    count = 0
    for x in xs:
        if x == 1:
            count += 1        
        else:
            if count: ys.append(count)
            count = 0
    if count: ys.append(count)
    return Counter(ys)

In [ ]:
count_runs([0, 1, 0, 1, 1, 0, 0, 0, 0, 1],)

In [ ]:
count_runs(np.random.randint(0,2,1000000))

**Part 2**

In [ ]:
def run_prob(expts, n, k, p):
    """Probability of observing at least one run of lenght k or more by simuulation."""
    xxs = np.random.choice([0,1], (expts, n), p=(1-p, p))
    return sum(max(d.keys()) >= k for d in map(count_runs, xxs))/expts

In [ ]:
run_prob(expts=100000, n=100, k=5, p=0.5)

In [ ]:
run_prob(expts=100000, n=100, k=7, p=0.7)

**Part 3**

In [9]:
from functools import lru_cache

@lru_cache()
def s(n, k, p):
    """Probability of observing at least one run of lenght k or more."""
    return sum(f(i, k, p) for i in range(1, n+1))

@lru_cache()
def f(n, k, p):
    """Helper function for s."""
    return u(n, k, p) - sum(f(i, k, p) * u(n-i, k, p) for i in range(1, n))

@lru_cache()
def u(n, k, p):
    """Helper function for f."""
    if n < k: 
        return 0
    return p**k - sum(u(n-i, k, p) * p**i for i in range(1, k))

In [10]:
s(100, 5, 0.5)

0.8101095991963579

In [11]:
s(100, 7, 0.7)

0.9491817984156692

**3**. (40 points)

Given matrix $M$
```python
      [[7, 8, 8],
       [1, 3, 8],
       [9, 2, 1]]
```

- Normalize the given matrix $M$ so that all rows sum to 1.0.  (5 points)
- The normalized matrix can then be considered as a transition matrix $P$ for a Markov chain. Find the stationary distribution of this matrix in the following ways using `numpy` and `numpy.linalg` (or `scipy.linalg`):
    - By repeated matrix multiplication of a random probability vector $v$ (a row vector normalized to sum to 1.0) with $P$ using matrix multiplication with `np.dot`. (5 points)
    - By raising the matrix $P$ to some large power until it doesn't change with higher powers (see `np.linalg.matrix_power`) and then calculating $vP$ (10 points)
    - From the equation for stationarity $wP = w$, we can see that $w$ must be a left eigenvector of $P$ with eigenvalue $1$ (Note: np.linalg.eig returns the right eigenvectors, but the left eigenvector of a matrix is the right eigenvector of the transposed matrix). Use this to find $w$ using `np.linalg.eig`. (20 points)

Suppose $w = (w_1, w_2, w_3)$. Then from $wP = w$, we have:
\begin{align}
w_1 P_{11} + w_2 P_{21} + w_3 P_{31} &= w_1 \\
w_1 P_{12} + w_2 P_{22} + w_3 P_{32} &= w_2 \\
w_1 P_{13} + w_2 P_{23} + w_3 P_{331} &= w_3 \\
\end{align}
This is a singular system, but we also know that $w_1 + w_2 + w_3 = 1$. Use these facts to set up a linear system of equations that can be solved with `np.linalg.solve` to find $w$.

**Part 1**

In [12]:
M = np.array([7,8,8,1,3,8,9,2,1.0]).reshape(3,3)
M

array([[7., 8., 8.],
       [1., 3., 8.],
       [9., 2., 1.]])

In [13]:
# Normalize the matrix so that rows sum to 1
P = M/np.sum(M, 1)[:, np.newaxis]
P

array([[0.30434783, 0.34782609, 0.34782609],
       [0.08333333, 0.25      , 0.66666667],
       [0.75      , 0.16666667, 0.08333333]])

**Part 2**

In [14]:
v = np.random.random(3)
v /= v.sum()
v

array([0.07208584, 0.47664814, 0.45126603])

In [15]:
# By raising the matrix $P$ to some large power 
P50 = np.linalg.matrix_power(P, 50)
P51 = np.dot(P50, P)
# check that P50 is stationary
np.testing.assert_allclose(P50, P51)
np.dot(v, P51)

array([0.39862184, 0.2605972 , 0.34078096])

**Part 3**

In [16]:
import scipy.linalg as la

In [17]:
lam, vec = la.eig(P, left=True, right=False)
idx = np.argmin(np.abs(lam - 1))
w = np.real(vec[:, idx])
w/w.sum()

array([0.39862184, 0.2605972 , 0.34078096])

In [18]:
# Left eigenvector with eigenvalue 1
# note transpose of P to find left eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(P.T) 
# find index of eigenvalue = 1 
idx = np.argmin(np.abs(eigenvalues - 1))
w = np.real(eigenvectors[:, idx]).T
# remember to normalize eigenvector to get a probability distribution
w/np.sum(w)

array([0.39862184, 0.2605972 , 0.34078096])

**Part 4**

We want to solve

\begin{align}
w_1 P_{11} + w_2 P_{21} + w_3 P_{31} &= w_1 \\
w_1 P_{12} + w_2 P_{22} + w_3 P_{32} &= w_2 \\
w_1 + w_2 + w_3 &= 1
\end{align}

In matrix form, we have

$$
\begin{pmatrix}
P_{11} - 1 & P_{21} & P_{31}\\
P_{12} & P_{22} -1 & P_{32} \\
1 & 1 & 1
\end{pmatrix}
\begin{pmatrix}
w_1 \\
w_2 \\
w_3
\end{pmatrix}
 = 
\begin{pmatrix}
0 \\ 0 \\ 1
\end{pmatrix}
$$

Knowing this, it is just a matter of setting up the system for `np.linalg.solve` to work.

In [19]:
# Solving linear system
# Use the first 2 rows of  the matrix P - I = 0
# and construnct the last row from $w_1 + w_2 + w_3 = 1$
A = P.T - np.eye(3)
A[2] = [1,1,1]
np.linalg.solve(A, [0,0,1])

array([0.39862184, 0.2605972 , 0.34078096])